In [3]:
import pandasdmx as sdmx
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash

In [4]:
ind_code = "HT_ADOL_UNMETMED_NOUNMET"

unicef = sdmx.Request("UNICEF")

metadata = unicef.dataflow("TRANSMONEE", provider="ECARO", version="1.0")


In [119]:
metadata.response.url

'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/dataflow/ECARO/TRANSMONEE/1.0?references=all'

In [5]:
metadata.response.from_cache

False

In [6]:
metadata

<pandasdmx.StructureMessage>
  <Header>
    id: 'IREF042403'
    prepared: '2022-02-11T18:30:48+00:00'
    receiver: <Agency not_supplied>
    sender: <Agency UNICEF>
    source: 
    test: False
  response: <Response [200]>
  Codelist (37): CL_INDICATORS_TRANSMONEE CL_CONF_STATUS CL_DECIMALS CL...
  ConceptScheme (3): CROSS_DOMAIN_CONCEPTS CS_SDG UNICEF_CONCEPTS
  DataflowDefinition (1): TRANSMONEE
  DataStructureDefinition (1): DSD_ECARO_TRANSMONEE
  AgencyScheme (2): AGENCIES DATA_PROVIDERS
  ProvisionAgreement (1): TRANSMONEE_ECARO_ECARO

In [7]:
metadata.response.url

'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/dataflow/ECARO/TRANSMONEE/1.0?references=all'

In [8]:
dataflows = sdmx.to_pandas(metadata.dataflow)
dataflows

TRANSMONEE    ECARO for TransMonEE
dtype: object

In [9]:
dsd = metadata.structure['DSD_ECARO_TRANSMONEE']


In [10]:
code = list(dsd.dimensions.get('INDICATOR').local_representation.enumerated)[0]

In [11]:
code.name.en


'Number of births (expressed in thousands)'

In [13]:
dsd.attributes.components

[<DataAttribute COVERAGE_TIME>,
 <DataAttribute UNIT_MEASURE>,
 <DataAttribute OBS_FOOTNOTE>,
 <DataAttribute FREQ>,
 <DataAttribute DATA_SOURCE>,
 <DataAttribute UNIT_MULTIPLIER>,
 <DataAttribute OBS_STATUS>]

In [14]:
dsd.measures.components

[<PrimaryMeasure OBS_VALUE>]

In [299]:
keys = {'INDICATOR': ["EDUNF_ADMIN_L2_MAT"]}

In [300]:
DEFAULT_DIMENSIONS = {
    "SEX": ["M", "F"],
    "RESIDENCE": ["U", "R"],
    "WEALTH_QUINTILE": ["Q1", "Q2", "Q3", "Q4", "Q5"]
}

In [301]:
dimensions = {}
keys.update(dimensions)
for key,value in DEFAULT_DIMENSIONS.items():
    keys[key] = value if key in keys and not keys[key] else ['_T']
    
keys

{'INDICATOR': ['EDUNF_ADMIN_L2_MAT'],
 'SEX': ['_T'],
 'RESIDENCE': ['_T'],
 'WEALTH_QUINTILE': ['_T']}

In [302]:
message = unicef.data(
    "TRANSMONEE", 
    provider="ECARO", 
    key={
        'INDICATOR': ['EDUNF_ADMIN_L2_MAT'],
        'SEX': ['_T'],
        'RESIDENCE': ['_T'],
        'WEALTH_QUINTILE': ['_T']
    }, 
    params = dict(startPeriod='2010', endPeriod='2021', lastNObservations=1), 
    dsd=dsd
)

/opt/homebrew/Caskroom/miniforge/base/envs/dash/lib/python3.9/site-packages/pandasdmx/api.py:260: UserWarning:

'provider' argument is redundant for <Resource.data: 'data'>



In [279]:
message.response.from_cache

AttributeError: 'Response' object has no attribute 'from_cache'

In [280]:
message.response.url

'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/TRANSMONEE/.EDUNF_GER_L1._T.._T._T?startPeriod=2010&endPeriod=2021&lastNObservations=1&labels=True'

In [ ]:
https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/TRANSMONEE/ALB+AND+ARM+AUT+AZE+BEL+BGR+BIH+BLR+CHE+CYP+CZE+DEU+DNK+ESP+EST+FIN+FRA+GBR+GEO+GRC+HRV+HUN+IRL+ISL+ITA+KAZ+KGZ+LIE+LTU+LUX+LVA+MCO+MDA+MKD+MLT+MNE+NLD+NOR+POL+PRT+ROU+RUS+SMR+SRB+SVK+SVN+SWE+TJK+TKM+TUR+UKR+UZB+VAT+XKX.EDUNF_OFST_L1+EDUNF_OFST_L2+EDUNF_OFST_L3._T._T._T._T?startPeriod=2010&endPeriod=2020&lastNObservations=1 

In [304]:
df = message.to_pandas(attributes="o", rtype="rows", dtype="str")

In [285]:
df.reset_index(inplace=True)

In [272]:
df.columns.names

FrozenList([None])

In [273]:
df.columns.levels

AttributeError: 'Index' object has no attribute 'levels'

In [107]:
df.index

DatetimeIndex(['2010-01-01', '2011-01-01'], dtype='datetime64[ns]', name='TIME_PERIOD', freq=None)

In [109]:
len(df.columns.levels)

6

In [114]:
df.columns = df.columns.droplevel(-1)

In [305]:
df

,,,,,,,value,UNIT_MEASURE,FREQ,DATA_SOURCE,UNIT_MULTIPLIER,OBS_STATUS
REF_AREA,INDICATOR,SEX,AGE,RESIDENCE,WEALTH_QUINTILE,TIME_PERIOD,,,,,,
AND,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,No,BINARY,1,UNESCO,0,A
ALB,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
ARM,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
AUT,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
AZE,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,No,BINARY,1,UNESCO,0,A
BIH,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
BEL,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
BGR,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A
BLR,EDUNF_ADMIN_L2_MAT,_T,_T,_T,_T,2018,Yes,BINARY,1,UNESCO,0,A


In [287]:
df["AGE"].unique()

array(['OVER_AGE', 'SCHOOL_AGE', 'UNDER_AGE'], dtype=object)

In [132]:
df.xs(('_T', '_T'), level=('SEX', 'RESIDENCE'), axis=1, drop_level=False)

REF_AREA                             AUT                                     \
INDICATOR       HT_ADOL_UNMETMED_NOUNMET                                      
SEX                                   _T                                      
AGE                               Y16T19 Y16T24 Y16T29 Y16T44 Y16T64 Y18T44   
RESIDENCE                             _T     _T     _T     _T     _T     _T   
WEALTH_QUINTILE                       _T     _T     _T     _T     _T     _T   
TIME_PERIOD                                                                   
2010-01-01                          96.3   97.0   97.2   97.2   97.4   97.1   
2011-01-01                          98.3   98.4   98.4   97.9   97.9   97.9   

REF_AREA                                     ...                      TUR  \
INDICATOR                                    ... HT_ADOL_UNMETMED_NOUNMET   
SEX                                          ...                       _T   
AGE             Y20T24 Y20T29 Y25T29 Y25T34  ...                   Y35T44   
RESIDENCE           _T     _T     _T     _T  ...                       _T   
WEALTH_QUINTILE     _T     _T     _T     _T  ...                       _T   
TIME_PERIOD                                  ...                            
2010-01-01        97.4   97.6   97.7   97.6  ...                     72.9   
2011-01-01        98.4   98.4   98.4   97.7  ...                     77.9   

REF_AREA                                                                        
INDICATOR                                                                       
SEX                                                                             
AGE             Y45T54 Y45T64 Y55T64 Y65T74 Y75T84 Y_GE16 Y_GE65 Y_GE75 Y_GE85  
RESIDENCE           _T     _T     _T     _T     _T     _T     _T     _T     _T  
WEALTH_QUINTILE     _T     _T     _T     _T     _T     _T     _T     _T     _T  
TIME_PERIOD                                                                     
2010-01-01        74.4   75.9   78.1   77.5   79.2   78.3   77.7   78.1   68.6  
2011-01-01        80.5   81.3   82.6   80.9   80.2   82.9   80.4   79.6   75.8  

[2 rows x 658 columns]

In [134]:
df.columns = df.columns.droplevel(-1)

In [135]:
df

REF_AREA                         AUT                                     \
INDICATOR   HT_ADOL_UNMETMED_NOUNMET                                      
SEX                               _T                                      
AGE                           Y16T19 Y16T24 Y16T29 Y16T44 Y16T64 Y18T44   
RESIDENCE                         _T     _T     _T     _T     _T     _T   
TIME_PERIOD                                                               
2010-01-01                      96.3   97.0   97.2   97.2   97.4   97.1   
2011-01-01                      98.3   98.4   98.4   97.9   97.9   97.9   

REF_AREA                                 ...                      TUR         \
INDICATOR                                ... HT_ADOL_UNMETMED_NOUNMET          
SEX                                      ...                       _T          
AGE         Y20T24 Y20T29 Y25T29 Y25T34  ...                   Y35T44 Y45T54   
RESIDENCE       _T     _T     _T     _T  ...                       _T     _T   
TIME_PERIOD                              ...                                   
2010-01-01    97.4   97.6   97.7   97.6  ...                     72.9   74.4   
2011-01-01    98.4   98.4   98.4   97.7  ...                     77.9   80.5   

REF_AREA                                                             
INDICATOR                                                            
SEX                                                                  
AGE         Y45T64 Y55T64 Y65T74 Y75T84 Y_GE16 Y_GE65 Y_GE75 Y_GE85  
RESIDENCE       _T     _T     _T     _T     _T     _T     _T     _T  
TIME_PERIOD                                                          
2010-01-01    75.9   78.1   77.5   79.2   78.3   77.7   78.1   68.6  
2011-01-01    81.3   82.6   80.9   80.2   82.9   80.4   79.6   75.8  

[2 rows x 658 columns]

In [194]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Graph(id='graph', figure=px.bar(df, x='REF_AREA', y='value', color='SEX', barmode='group', title='Indicator')),
    
])
# # Define callback to update graph
# @app.callback(
#     Output('graph', 'figure'),
#     [Input("colorscale-dropdown", "value")]
# )
# def update_figure(colorscale):
#     return px.scatter(
#         df, x="total_bill", y="tip", color="size",
#         color_continuous_scale=colorscale,
#         render_mode="webgl", title="Tips"
#     )
# Run app and display result inline in the notebook
del app.config._read_only["requests_pathname_prefix"]
app.run_server(mode='inline')

/opt/homebrew/Caskroom/miniforge/base/envs/dash/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

